# TODO: the following 5 functions for CRUD(create, read,update,delete) operations that work with sqlite database called chinook.db

* def create_connection(dbpath):
*   * can add verbose parameter that prints sqlite version used
*   return conn
* def create_artist(conn, artist_name):
   * use parametrized query !
*   * do not have to return anything but can use try: inside this function
* def read_artists(conn):  * can add some extra parameters to limit
*   return artists  * can return a list of tuples, or you can create a list of artist objects if you want
* def update_artist(id, new_name):
* def delete_artist(id=None, name=""):
*    * provide deletion by id AND/OR name


* test it by adding you name to artists table smile

# you can use .ipynb or .py file

In [2]:
import sqlite3
import pandas as pd


In [ ]:
class SqliteCRUD:
    
    # called when we create an object of this class
    def __init__(self, dbpath):
        self.dbpath = dbpath
        self.conn = sqlite3.connect(dbpath) # create a connection to database
        self.cursor = self.conn.cursor() # create cursor to make queries
         
        print(f"Connected to database: {dbpath} Ready to perform CRUD operations!")
    
    # destructor - will be called when object is deleted - typically when program ends
    def __del__(self):
        print("Closing connection")
        self.conn.close()
    
    # get a list of all artists
    def read_artists(self):
        self.cursor.execute("SELECT * FROM artists")
        return self.cursor.fetchall() # returns a list of tuples

    def create_artist(self, name):
        # notice the parameterized query - for security reasons
        self.cursor.execute("INSERT INTO artists (name) VALUES (?)", (name,))
        self.conn.commit()
        # return self to allow chaining
        return self

    # insert a list of artists
    def create_artists(self, artists):
        self.cursor.executemany("INSERT INTO artists (name) VALUES (?)", artists)
        # alternative:
        # for artist in artists:
        #     self.cursor.execute("INSERT INTO artists (name) VALUES (?)", artist)
        # should be similar speed as long as we do not have too many artists
        # also note how we only commit once
        self.conn.commit()
        return self

    # update artist name
    def update_artist_by_id(self, id, new_name):
        # again parameterized query for security reasons
        self.cursor.execute("UPDATE artists SET name = ? WHERE ArtistId = ?", (new_name, id))
        self.conn.commit()
        return self

    # update arits name by name
    def update_artist_by_name(self, old_name, new_name):
        # again parameterized query for security reasons
        # this could change multiple rows !
        self.cursor.execute("UPDATE artists SET name = ? WHERE name = ?", (new_name, old_name))
        self.conn.commit()
        return self

    # let's get tail of artists table
    def read_artists_tail(self, n=10):
        self.cursor.execute("SELECT * FROM artists ORDER BY ArtistId DESC LIMIT ?", (n,))
        return self.cursor.fetchall()

    # delete artist by id
    def delete_artist_by_id(self, id):
        # again parameterized query for security reasons
        self.cursor.execute("DELETE FROM artists WHERE ArtistId = ?", (id,))
        self.conn.commit()
        return self

    # delete artist by name
    def delete_artist_by_name(self, name):
        # again parameterized query for security reasons
        self.cursor.execute("DELETE FROM artists WHERE name = ?", (name,))
        self.conn.commit()
        return self

    # return dataframe with all artists
    def read_artists_df(self):
        # by abstracting this we can change the implementation later
        # also we create documentation for this method
        return pd.read_sql_query("SELECT * FROM artists", self.conn)

    def read_albums_df(self):
        return pd.read_sql_query("SELECT * FROM albums", self.conn)

    def read_tracks_df(self):
        return pd.read_sql_query("SELECT * FROM tracks", self.conn)

    def read_album_stats(self):
        query = """
        --how about getting total running length of each album?
        SELECT
            t.albumid,
            a.Title AlbumTitle,
            a2.Name ArtistName,
            COUNT(trackid),
            SUM(Milliseconds)/(1000*60) AlbumLengthMinutes,
            AVG(Milliseconds)/(1000*60) AverageSongLengthMin,
            MIN(Milliseconds)/(1000*60) ShortestSongMin,
            MAX(Milliseconds)/(1000*60) LongestSongMin
        FROM
            tracks t
        JOIN albums a 
        ON a.AlbumId = t.AlbumId 
        JOIN artists a2 
        ON a.ArtistId = a2.ArtistId 
        GROUP BY
            t.albumid
        --SO HAVING is similar to WHERE but we use it in GROUP BY situations
        -- disabling HAVING it for DF since we can filter there 
        -- HAVING AlbumLengthMinutes < 80
        ORDER BY AlbumLengthMinutes DESC;
        """
        return pd.read_sql_query(query, self.conn)

In [ ]:
# we create a single object of this class
db = SqliteCRUD("chinook.db")
db.dbpath

In [ ]:
artist_list = db.read_artists()
artist_list[:3]